
# Title: [SW-Hierarchy]: [BOM data extraction]
### Author: [Erick Rodriguez Martinez | +52 452 159 07 12 | +1 653 325 81 15 | erodriguezm91@outlook.com]
### Date: [04/14/2023 - Last Update]
### Template: [v1.0 | 04142023 - First Release]
### Description: [SW-Hierarchy is a python module for extracting SW-BOM data and sorting it to generate custom BOMs as follows:]
* #### CPL - Comercial Parts List
* #### MPL - Manufacturing Parts List
* #### RAW - RAW Bill of Materials
---


# [PIP] Python Installation Packages

In [18]:
# xlrd is a library for reading data and formatting information from Excel files in the historical ".xls" format.
#@TODO Remove "#" to install "xlrd"
#!pip install xlrd
#!pip install openpyxl

# An open-source library that provides high-performance data manipulation in Python
#@TODO Remove "#" to install "pandas"
#!pip install pandas

# Python library dedicated to reading, manipulating and generating PDF documents
#@TODO Remove "#" to install "borb"
#!pip install borb


# [ISL] Import Standard Libraries

In [19]:
#TODO [OPTIONAL Remove if not used]


# [IT-PL] Import Third-Party Libraries

In [20]:
import pandas as pd
from pandas import ExcelWriter

import numpy as np


# [T-LI] TEST Library Imports

In [21]:
#TODO Remove if not used or before to release version
from decimal import Decimal

from borb.pdf import MultiColumnLayout
from borb.pdf import SingleColumnLayoutWithOverflow
from borb.pdf import PageLayout
from borb.pdf import TableUtil
from borb.pdf import TableCell

from borb.pdf import FlexibleColumnWidthTable
from borb.pdf import Paragraph
from borb.pdf import Document
from borb.pdf import Page
from borb.pdf import PDF
from borb.pdf.page.page_size import PageSize


# [IIM] Import internal modules

In [22]:
#TODO Remove if not used or before to release version


# [DGC] Define Global Constants


# [DGV] Define Global Variables

In [23]:
#TODO [OPTIONAL Remove if not used]



# [FNC] Functions

In [24]:
def f_clean_nan_values(dataFrame, column, string):
    """
    This function finds all NaNs in the #Column of a DataFrame and replaces them with "string" value.

    Args:
        dataFrame (): Panda *.xlsx read file.
        column (int): Number of the column where "type" is found.
        string (str): NaN values founded in "Type" cells would be replaced by str1.

    Returns:
        dataFrame:
    """

    _dataFrame = dataFrame
    _dataFrame.iloc[:, column] = _dataFrame.iloc[:, column].replace(np.nan, string)

    return _dataFrame


def f_hierarchy_quantity_update(dataFrame, hierarchyCol, quantityCol):
    """
    This function uses the hierarchy to calculate the real quantity value to update the DataFrame.

    Args:
        dataFrame (df): Panda *.xlsx read file.
        hierarchyCol (int): Number of the column where "hierarchy" is found.
        quantityCol (int): Number of the column where "quantity" is found.

    Returns:
        dataFrame:
    """
    _dataFrame = dataFrame

    row, column = _dataFrame.shape
    h = [0] * row
    c = [0] * row

    for i in range(len(_dataFrame.iloc[:, hierarchyCol])):
        # print(len(_dataFrame.iloc[:,hierarchyCol])) # Displays the total number of rows inside _dataFrame

        hierarchy = 1
        for j in str(_dataFrame.iat[i, hierarchyCol]):
            # print(j) #TODO Remove "#" to show the 'j' character
            if j == '.':
                hierarchy += 1
        # print(hierarchy) #TODO Remove "#" to debug "Hierarchy" value

        quantityValue = _dataFrame.iat[i,quantityCol]
        sum = 0
        if hierarchy == 1:
            h[1] = quantityValue
            c[i] = h[1]
        else:
            h[hierarchy] = quantityValue
            c[i] = h[hierarchy]
            for j in range(hierarchy):
                if j == 0:
                    continue
                else:
                    c[i] = c[i] * h[j]
    # print(c) #@TODO Remove "#" to debug Quantity -[c]- Output

    for i in range(len(c)):
        _dataFrame.iloc[i,quantityCol] = c[i]

    return _dataFrame


def f_split_data_frame(dataFrame, colN, str1):
    """
    This function finds all blanks and removes them from the string..

    Args:
        dataFrame (): Panda *.xlsx read file.
        column (int): Number of the column where to look for blanks.

    Returns:
        dataFrame: A DataFrame with no blank spaces in the selected column
    """
    _dataFrame = dataFrame
    colSelect = _dataFrame[_dataFrame.columns[colN]]
    _dataFrame = _dataFrame[colSelect == str1]
    return  _dataFrame


def f_clean_blank_spaces(dataFrame, column):
    """
    This function finds all blanks and removes them from the string..

    Args:
        dataFrame (): Panda *.xlsx read file.
        column (int): Number of the column where to look for blanks.

    Returns:
        dataFrame: A DataFrame with no blank spaces in the selected column
    """
    _dataFrame = dataFrame

    for i in range(len(_dataFrame.iloc[:, column])):
        _dataFrame.iloc[i,column] = str(_dataFrame.iat[i,column]).replace(' ', '')

    return _dataFrame


def f_unique_values(dataFrame):
    """
    This function finds all UniqueValues of the selected column in a DataFrame.

    Args:
        dataFrame (): Panda *.xlsx read file.
        column (int): Number of the column where to look for blanks.

    Returns:
        dataFrame: A DataFrame with Unique Values
    """
    _dataFrame = dataFrame
    row, col = _dataFrame.shape

    uniqueValues = _dataFrame.iloc[:, 1] # Select All rows (:) from column one (1) of the DataFrame
    uniqueValues = uniqueValues.unique() # UniqueValues founded in the selected column


    # c = [0] * row #TODO Debug to delete this line, could be not used
    foundValuesArray = [0] * len(uniqueValues) # Variable to store the UniqueValues founded
    quantityArray = [0] * len(uniqueValues) # Variable to store the accumulative quantity of the UniqueValues
    iteration = 0 #TODO Debug to delete this line, could be not used
    indexToDeleteArray = [0] * row
    uniqueDataFrame = _dataFrame

    # For loop to explore all rows of the DataFrame
    for i in range(row):
        iteration = 0
        # print(i) #TODO Remove "#" to debug Iteration
        for j in uniqueValues: # For loop to search for occurrences of UniqueValues vs the rows values of the DataFrame
            if j == str(_dataFrame.iat[i,1]):
                if foundValuesArray[iteration] == 0:
                    # foundValueArray[iteration] = 1 #TODO Debug to delete this line, could be not used
                    quantityArray[iteration] += int(_dataFrame.iat[i,3])
                else:
                    quantityArray[iteration] += int(_dataFrame.iat[i,3])
                    indexToDeleteArray[i] = _dataFrame.index.values[i] # Adds the current index of the occurrence to the array containing the indexes to be deleted
                foundValuesArray[iteration] = 1 #TODO Debug to delete this line, could be not used
                # print(foundValuesArray) #TODO Remove "#" to debug foundValuesArray
            iteration += 1

    # For loop to delete duplicate values using the indexes stored in the IndexToDeleteArray
    for i in indexToDeleteArray:
        if i != 0:
            uniqueDataFrame = uniqueDataFrame.drop(i)

    # For loop to update the Quantity value of the UniqueValues
    iteration = 0
    for i in quantityArray:
        uniqueDataFrame.iat[iteration,3] = i
        iteration += 1

    print(uniqueValues) #TODO Remove "#" to debug Unique Values
    #print('Accumulative quantity array')
    #print(quantityArray)
    #print('Indexes to be deleted array')
    #print(indexToDeleteArray)

    return uniqueDataFrame


def f_clearNaN(dataFrame, col1):
    _dataFrame = dataFrame

    for i in range(len(_dataFrame.iloc[:, col1])):
        _dataFrame.iloc[:, col1] = _dataFrame.iloc[:, col1].replace(np.nan, 'N/A')
    return  _dataFrame


def f_rawMaterial(dataFrame, colSelect, sort1, sort2, colEvaluate):
    """
    This function .

    Args:
        dataFrame (): Panda *.xlsx read file.
        colSelect (int): Number of the column where to look for blanks.
        sort1 (type):
        sort2 (type):
        colEvaluate (type):

    Returns:
        dataFrame: A DataFrame with Unique Values
    """
    _dataFrame = dataFrame
    _dataFrame = _dataFrame[_dataFrame.columns[colSelect]]
    columnValues = _dataFrame.columns.values.tolist()
    #print(columnValues)

    _dataFrame = _dataFrame.sort_values(by=columnValues[sort1])
    _dataFrame = _dataFrame.sort_values(by=columnValues[sort2])

    _dataFrame = _dataFrame[_dataFrame[columnValues[colEvaluate]] > 0]
    return _dataFrame


# [T-FNC] TEST Functions

In [26]:
def f_uniqueRawQuantity(dataFrame):
    _dataFrame = dataFrame
    row, col = _dataFrame.shape

    testValues1 = _dataFrame.iloc[:, 1]
    testValues1 = testValues1.unique()
    print(testValues1)

    testValues2 = _dataFrame.iloc[:, 2]
    testValues2 = testValues2.unique()
    print(testValues2)

    c = [0] * row
    found = np.zeros((len(testValues1), len(testValues2)))

    quantity = np.zeros((len(testValues1), len(testValues2)))

    rowToDelete = [0] * row

    uniqueDataFrame = _dataFrame
    selectCols = _dataFrame.columns

    for i in range(row):
        i_iteration = 0
        j_iteration = 0
        k_iteration = 0
        #print(i)
        for j in testValues1:
            if j == str(_dataFrame.iat[i,1]):
                for k in testValues2:
                    if k == str(_dataFrame.iat[i,2]):
                        #print(str(j_iteration), str(k_iteration))
                        if _dataFrame.iat[i,4] > 0:
                            quantity[j_iteration, k_iteration] += int(_dataFrame.iat[i,0]) * _dataFrame.iat[i,3] * _dataFrame.iat[i,4]
                            #print(quantity[j_iteration, k_iteration])
                        else:
                            quantity[j_iteration, k_iteration] += int(_dataFrame.iat[i,0]) * _dataFrame.iat[i,3]
                            #print(quantity[j_iteration, k_iteration])
                        found[j_iteration,k_iteration] = 1
                    k_iteration += 1
            j_iteration += 1

    print(found)
    #print(quantity)
    df = pd.DataFrame(columns=['Material', 'Presentacion', 'Total'])
    row, col = found.shape
    for i in range(row):
        for j in range(col):
            if found[i,j] != 0:
                i_data = testValues1[i]
                j_data = testValues2[j]
                toApend = pd.Series([i_data, j_data, quantity[i,j]], index=['Material', 'Presentacion', 'Total'])
                print(toApend)

                #df.concat(other=[i_data, j_data], ignore_index=True)
                #pd.concat(df, i_data, j_data)
                df = pd.concat([df, toApend.to_frame().T], ignore_index=True)
    return df


# [C] Classes

class MyClass:
    """
    Class description.
    """
    def __init__(self, arg1, arg2):
        """
        Class constructor.

        Args:
            arg1 (type): Description of arg1.
            arg2 (type): Description of arg2.
        """
        self.arg1 = arg1
        self.arg2 = arg2

    def method1(self):
        """
        Method description.
        """
        # Method logic

    def method2(self):
        """
        Method description.
        """
        # Method logic



# [TC] TEST Classes

# [PF] Program Flow

In [27]:
#TODO *.xlsx Source File Path
xlsxPath = 'AA02A001.xlsx'
xlsxSrcToDF = pd.read_excel(xlsxPath)

#TODO Set The Output File Name
xlsxName = xlsxPath.replace('.xlsx', '_M.xlsx')

# Clean NaN Values of The "Type" Column
nanCleanDF = f_clean_nan_values(xlsxSrcToDF, 0, 'C')

# Quantity Update by Hierarchy & Delete Hierarchy Column
quantityUpdatedDF = f_hierarchy_quantity_update(nanCleanDF, 1, 4)
quantityUpdatedDF = quantityUpdatedDF.drop(quantityUpdatedDF.columns[1], axis=1)

# Split DataFrame in Manufacturing BOM & Clean Blank Spaces
manufacturingDF = f_split_data_frame(quantityUpdatedDF, 0, 'F')
manufacturingDF = f_clean_blank_spaces(manufacturingDF, 1)
manufacturingToXLSX = f_unique_values(manufacturingDF)

# Split DataFrame in Comercial BOM & Clean Blank Spaces
comercialDF = f_split_data_frame(quantityUpdatedDF, 0, 'C')
comercialDF = f_clean_blank_spaces(comercialDF, 1)
comercialToXLSX = f_unique_values(comercialDF)

['AA02P031' 'AA02S021' 'AA02P035' 'AA02P036' 'AA02P038' 'AA02S013'
 'AA02S002' 'AA02P003' 'AA02P002' 'AA02S001' 'AA02P001' 'AA02S003'
 'AA02P004' 'AA02S012' 'AA02S004' 'AA02P005' 'AA02P006' 'AA02P007'
 'AA02S005' 'AA02P008' 'AA02S006' 'AA02P009' 'AA02P010' 'AA02S007'
 'AA02P011' 'AA02S008' 'AA02P012' 'AA02P013' 'AA02P014' 'AA02S009'
 'AA02P015' 'AA02P016' 'AA02P017' 'AA02S010' 'AA02P018' 'AA02S011'
 'AA02P019' 'AA02S016' 'AA02P022' 'AA02P024' 'AA02P023' 'AA02P025'
 'AA02P026' 'AA02P027' 'AA02S015' 'AA02S014' 'AA02P020' 'AA02P021'
 'AA02S019' 'AA02P030' 'AA02S018' 'AA02S017' 'AA02P028' 'AA02P029'
 'AA02S020' 'AA02P032' 'AA02P033' 'AA02S022' 'AA02P039' 'AA02P040']
['RLW0.25-AC-G' 'HNUTG20.2500-20-D-N-AC-G'
 'RHSNBOLT0.25-20x0.75x0.75-N-AI-G' 'REM0.125-0.5-AI' 'HC63-8.625'
 'HNUTG20.5000-13-D-N-AC-G' 'HBOLTG20.5000-13x5x5-N-AC-G'
 'RHSNBOLT0.375-16x1x1-N-AC-G' 'RLW0.375-AC-G' 'HNUTG20.3750-16-D-N-AC-G'
 'AA02P034' 'RNYMS1-1420YA-40' 'UCP205-16' 'RFW0.5-AC-G'
 'HBOLTG20.5000-13x6x1.25-C-AC

# [TPF] Test Program Flow


In [28]:
rawMaterial = f_rawMaterial(xlsxSrcToDF, [4, 5, 6, 7, 8, 9], 1, 2, 3)
rawMaterial = f_clearNaN(rawMaterial,2)
rawMaterial

,Cant.,Material,Presentación,Largo,Ancho,Peso
61,2,ASTM A36 Acero,"Cuadrado 0.25"" x 0.25""",31.75,NaN,0.01
63,3,ASTM A36 Acero,"Cuadrado 0.25"" x 0.25""",76.20,NaN,0.02
64,1,ASTM A36 Acero,"Cuadrado 0.25"" x 0.25""",138.68,NaN,0.04
62,1,AISI 430 A. I.,Lám. Cal. 10,318.48,158.00,0.63
23,1,AISI 430 A. I.,Lám. Cal. 10,247.61,82.54,0.42
18,4,AISI 430 A. I.,Lám. Cal. 10,76.20,76.20,0.12
40,1,AISI 430 A. I.,Lám. Cal. 10,247.61,82.54,0.42
65,2,AISI 430 A. I.,Lám. Cal. 10,31.75,31.75,0.02
43,6,AISI 430 A. I.,Lám. Cal. 12,101.60,38.10,0.07
35,1,AISI 430 A. I.,Lám. Cal. 12,1371.60,169.69,4.56


In [29]:
values = rawMaterial.iloc[:,1]
values = values.unique()
values

array(['ASTM A36 Acero', 'AISI 430 A. I.', 'AISI 1018 Acero', 'A. C.',
       'PVC'], dtype=object)

In [30]:
values2 = rawMaterial.iloc[:,2]
values2 = values2.unique()
values2

array(['Cuadrado 0.25" x 0.25"', 'Lám. Cal. 10', 'Lám. Cal. 12',
       'Lám. Cal. 14', 'PTR Cuadrado 1.5" x 1.5" Cal. 14',
       'PTR Cuadrado 2" x 2" Cal. 14', 'Placa Esp. 3/16"',
       'Redondo de 0.75"', 'Redondo de 1"', 'Redondo de 2.5"', 'N/A'],
      dtype=object)

In [31]:
xlsxRawMaterialUnique = f_uniqueRawQuantity(rawMaterial)
xlsxRawMaterialUnique

['ASTM A36 Acero' 'AISI 430 A. I.' 'AISI 1018 Acero' 'A. C.' 'PVC']
['Cuadrado 0.25" x 0.25"' 'Lám. Cal. 10' 'Lám. Cal. 12' 'Lám. Cal. 14'
 'PTR Cuadrado 1.5" x 1.5" Cal. 14' 'PTR Cuadrado 2" x 2" Cal. 14'
 'Placa Esp. 3/16"' 'Redondo de 0.75"' 'Redondo de 1"' 'Redondo de 2.5"'
 'N/A']
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Material                ASTM A36 Acero
Presentacion    Cuadrado 0.25" x 0.25"
Total                           430.78
dtype: object
Material         ASTM A36 Acero
Presentacion    Redondo de 2.5"
Total                     254.0
dtype: object
Material        AISI 430 A. I.
Presentacion      Lám. Cal. 10
Total              116437.1838
dtype: object
Material        AISI 430 A. I.
Presentacion      Lám. Cal. 12
Total              3113278.762
dtype: object
Material        AISI 430 A. I.
Presentacion      Lám. Cal. 14
Total          

,Material,Presentacion,Total
0,ASTM A36 Acero,"Cuadrado 0.25"" x 0.25""",430.78
1,ASTM A36 Acero,"Redondo de 2.5""",254.0
2,AISI 430 A. I.,Lám. Cal. 10,116437.1838
3,AISI 430 A. I.,Lám. Cal. 12,3113278.762
4,AISI 430 A. I.,Lám. Cal. 14,812159.722
5,AISI 430 A. I.,"PTR Cuadrado 1.5"" x 1.5"" Cal. 14",6437.0
6,AISI 430 A. I.,"PTR Cuadrado 2"" x 2"" Cal. 14",20430.2
7,AISI 430 A. I.,"Placa Esp. 3/16""",28460.9946
8,AISI 430 A. I.,"Redondo de 0.75""",8.0
9,AISI 1018 Acero,"Redondo de 1""",3775.0


In [32]:
writer = ExcelWriter(xlsxName)

xlsxSrcToDF = xlsxSrcToDF.reset_index(drop=True)
xlsxSrcToDF.to_excel(writer, 'Source')

manufacturingToXLSX = manufacturingToXLSX.reset_index(drop=True)
manufacturingToXLSX.to_excel(writer, 'Manufacture')

comercialToXLSX = comercialToXLSX.reset_index(drop=True)
comercialToXLSX.to_excel(writer, 'Comercial')

xlsxRawMaterialUnique.to_excel(writer, 'RawMaterials')


writer.save()
writer.close()

C:\Users\erodr\AppData\Local\Temp\ipykernel_43324\3350119021.py:15: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [33]:
doc = Document()
page = Page(PageSize.TABLOID_LANDSCAPE.value[0], PageSize.TABLOID_LANDSCAPE.value[1],)

doc.add_page(page)
pageLayOut = SingleColumnLayoutWithOverflow(page)
pageLayOut._vertical_margin_top = page.get_page_info().get_height() * Decimal(0.02)
pageLayOut._vertical_margin_bottom = page.get_page_info().get_height() * Decimal(0.02)
pageLayOut._horizontal_margin = page.get_page_info().get_width() * Decimal(0.01)

rows, columns = manufacturingDF.shape
xlsxManufactureArray = manufacturingDF.to_numpy()


In [34]:

xlsxManufactureArray.

SyntaxError: invalid syntax (1746554950.py, line 1)

In [ ]:
from borb.pdf.canvas.color.color import HexColor, X11Color
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable as Table
#from borb.pdf.canvas.layout.table.flexible_column_width_table import FlexibleColumnWidthTable as Table
rows, columns = manufacturingDF.shape
columnsDF = manufacturingDF.columns.to_numpy()

manufactureTable = Table(number_of_rows = rows, number_of_columns = columns)
for h in  columnsDF:
    manufactureTable.add(
        TableCell(
            Paragraph(h, font_color=X11Color("White")),
            background_color=HexColor("016934")
        )
    )
print(xlsxManufactureArray.shape)
data = 0
for i in iter(xlsxManufactureArray):
    print(i)
    for j in range(len(i):
        print(j)
        print(i[j])
        data = i[j]
        print(data)
        manufactureTable.add(Paragraph(data))





pageLayOut.add(manufactureTable)

with open("output.pdf", "wb") as out_file_handle:
    PDF.dumps(out_file_handle, doc)

In [ ]:
x
pageLayOut.add(
    #TableUtil.from_2d_array(xlsxManufactureArray,flexible_column_width=True)
    #TableCell.

    )
# store
with open("output.pdf", "wb") as out_file_handle:
    PDF.dumps(out_file_handle, doc)

In [ ]:
xlsxRawMaterial = xlsxSrcToDF[xlsxSrcToDF.columns[[4, 5, 6, 7, 8]]]
srie = xlsxRawMaterial.columns.values.tolist()
print(srie)

xlsxRawMaterial = xlsxRawMaterial.sort_values(by=srie[1])
xlsxRawMaterial = xlsxRawMaterial.sort_values(by=srie[2])

xlsxRawMaterial = xlsxRawMaterial[xlsxRawMaterial[srie[3]] > 0]
xlsxRawMaterial

In [ ]:
                if found[i_iteration] == 0:
                    found[i_iteration] = 1
                    quantity[i_iteration] += int(_dataFrame.iat[i,3])
                else:
                    quantity[i_iteration] += int(_dataFrame.iat[i,3])
                    rowToDelete[i] = _dataFrame.index.values[i]
                found[i_iteration] = 1
                print(found)
            i_iteration += 1

    for i in rowToDelete:
        if i != 0:
            uniqueDataFrame = uniqueDataFrame.drop(i)
            #print(j)

    print('Result')
    print(found)
    print(quantity)
    print(rowToDelete)

    i_iteration = 0
    for i in quantity:
        print(i)
        print(i_iteration)
        uniqueDataFrame.iat[i_iteration,3] = i
        i_iteration += 1